In [1]:
!pip install -q selenium undetected-chromedriver pandas tqdm
!wget -q -O chrome.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt-get update -y > /dev/null
!apt-get install -y libvulkan1 libnss3 libxss1 libgconf-2-4 libasound2 > /dev/null
!dpkg -i chrome.deb
!apt-get -f install -y > /dev/null

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 26.1 MB/s eta 0:00:00
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Selecting previously unselected package google-chrome-stable.
(Reading database ... 126516 files and directories currently installed.)
Preparing to unpack chrome.deb ...
Unpacking google-chrome-stable (135.0.7049.95-1) ...
Setting up google-chrome-stable (135.0.7049.95-1) ...
update-alternatives: using /usr/bin/google-chrome-stable to provide /usr/bin/x-www-browser (x-www-browser) in auto mode
update-alternatives: using /usr/bin/google-chrome-stable to provide /usr/bin/gnome-www-browser (gnome-www-browser) in auto mode
update-

In [2]:
!google-chrome --version

Google Chrome 135.0.7049.95 


In [3]:
import os
import re
import time
import pandas as pd
import requests
from tqdm import tqdm
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor, as_completed

import shutil
import uuid
from google.colab import files

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### Upload "twigema_release.csv“ (Expand Files at left side) ###

In [24]:
INPUT_CSV_PATH = "/content/twigma_release.csv"   # Upload the twigma_release.csv

### Set the Start and End for partial process ###

In [52]:
START_ROW = 1000
END_ROW = 2000  # Set your own range

OUTPUT_IMAGE_DIR = f"/content/images_{START_ROW}_{END_ROW}"
os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

# Load and filter single-image tweets
df_all = pd.read_csv(INPUT_CSV_PATH)
df_all["tweet_id"] = df_all["id"].astype(str)
df_chunk = df_all.iloc[START_ROW:END_ROW].copy()

tweet_counts = df_chunk["tweet_id"].value_counts()
single_tweet_ids = tweet_counts[tweet_counts == 1].index
df = df_chunk[df_chunk["tweet_id"].isin(single_tweet_ids)].copy()
print(f"✅ Processing {len(df)} single-image tweets only")

# Launch headless Chrome browser
options = uc.ChromeOptions()
options.binary_location = "/usr/bin/google-chrome"
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = uc.Chrome(options=options)

def restart_driver():
    global driver
    try:
        driver.quit()
    except:
        pass

    options = uc.ChromeOptions()
    options.binary_location = "/usr/bin/google-chrome"
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = uc.Chrome(options=options)
    print("♻️ Driver restarted.")


✅ Processing 568 single-image tweets only


### Start Crawl ###

In [53]:
cdn_names = []
tweet_texts = []
image_success = []
text_success = []
row_indices = []

actual_requests = 0
sequent_requests = 0
consecutive_failures = 0
ALL_consecutive_failures = 0
duration = 0
recent_texts = []
recent_images = []

start_time = time.time()

for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing tweets"):
    tweet_id = row["tweet_id"]
    tweet_url = f"https://twitter.com/i/web/status/{tweet_id}"
    row_index = row.name  # Original row number in the TWIGMA dataset

    media_url = ""
    success = False
    tweet_text = ""
    text_ok = False


    try:
        driver.get(tweet_url)
        actual_requests += 1  # ✅ Actual browser requests
        sequent_requests += 1  # ✅ Sequential browser requests
        # time.sleep(1)

        # ✅ Extract tweet text
        try:
            tweet_elem = WebDriverWait(driver, 2.5).until(
                EC.presence_of_element_located((By.XPATH, '//article//div[@data-testid="tweetText"]'))
            )
            tweet_text = tweet_elem.text.strip()
            text_ok = True
        except:
            print(f"\n[WARN] ❌ Failed to extract tweet text: {tweet_id}")

        # ✅ Extract image URL
        try:
            img_elem = WebDriverWait(driver, 0.5).until(
                EC.presence_of_element_located((By.XPATH, '//article//img[contains(@src, "pbs.twimg.com/media")]'))
            )
            media_url = img_elem.get_attribute("src")
        except:
            print(f"\n[WARN] ❌ Failed to locate image element: {tweet_id}")

        # ✅ Download image (record URL even if download fails)
        if media_url:
            try:
                parsed = urlparse(media_url)
                cdn_filename = os.path.basename(parsed.path)
                if not re.search(r"\.(jpg|jpeg|png|webp)$", cdn_filename):
                    cdn_filename += ".jpg"

                full_filename = f"{tweet_id}_{cdn_filename}"
                full_path = os.path.join(OUTPUT_IMAGE_DIR, full_filename)

                img_data = requests.get(media_url, timeout=5).content
                with open(full_path, "wb") as f:
                    f.write(img_data)

                cdn_names.append(cdn_filename)
                success = True  # ← download successful

            except Exception as e:
                print(f"[WARN] 🌐 Image download failed: {tweet_id} → {media_url}")
                cdn_names.append(media_url)  # Save image URL for retry

        else:
            cdn_names.append("")

    except Exception as e:
        print(f"[ERROR] 🚨 Tweet {tweet_id} failed: {e}")
        cdn_names.append("")
        tweet_text = ""
        text_ok = False

    recent_texts.append(text_ok)
    recent_images.append(success)

    # ✅ Record result for each tweet
    tweet_texts.append(tweet_text)
    text_success.append(text_ok)
    image_success.append(success)
    row_indices.append(row_index)

    # ===
    if actual_requests % 10 == 0:
        text_count = sum(recent_texts)
        image_count = sum(recent_images)
        both_count = sum(t and i for t, i in zip(recent_texts, recent_images))

        print(f"\n📊 Summary of the last 10 requests:")
        print(f"📝 Text success:  {text_count}/10")
        print(f"🖼️  Image success: {image_count}/10")
        print(f"✅ Both success:  {both_count}/10\n")

        recent_texts = []
        recent_images = []

    # === Cooling logic ===
    if not text_ok and not success:
        consecutive_failures += 1
        ALL_consecutive_failures += 1
    else:
        consecutive_failures = 0
        ALL_consecutive_failures = 0

    # === Restart driver every 10 seqential tweets
    if sequent_requests >= 10:
        print(f"♻️ Restarting driver after {sequent_requests} requests...")
        restart_driver()
        time.sleep(15)
        consecutive_failures = 0
        sequent_requests = 0

    # === Restart driver after 5 consecutive failures
    elif consecutive_failures >= 5:
        print(f"♻️ Restarting driver after {consecutive_failures} consecutive failures...")
        restart_driver()
        time.sleep(15)
        consecutive_failures = 0
        sequent_requests = 0

        if ALL_consecutive_failures >= 10:
            restart_driver()
            time.sleep(15)
            consecutive_failures = 0
            print(f"ALL_consecutive_failures: {ALL_consecutive_failures}")




end_time = time.time()
duration = end_time - start_time

Processing tweets:   0%|          | 0/568 [00:00<?, ?it/s]


[WARN] ❌ Failed to extract tweet text: 1608604029759942657


Processing tweets:   0%|          | 1/568 [00:06<59:28,  6.29s/it]


[WARN] ❌ Failed to locate image element: 1608604029759942657


Processing tweets:   0%|          | 2/568 [00:09<40:14,  4.27s/it]

...

Processing tweets: 100%|█████████▉| 566/568 [1:00:11<00:07,  3.92s/it]


[WARN] ❌ Failed to locate image element: 1608250728203649024


Processing tweets: 100%|█████████▉| 567/568 [1:00:14<00:03,  3.70s/it]


[WARN] ❌ Failed to extract tweet text: 1608249104462643200

[WARN] ❌ Failed to locate image element: 1608249104462643200
♻️ Restarting driver after 10 requests...
♻️ Driver restarted.


Processing tweets: 100%|██████████| 568/568 [1:00:35<00:00,  6.40s/it]


### Saving CSV & Summary###

In [54]:
# ✅ Save results to CSV
df["tweet_text"] = tweet_texts
df["cdn_image_name"] = cdn_names
df["text_success"] = text_success
df["image_success"] = image_success
df["twigma_row"] = row_indices

OUTPUT_CSV_PATH = f"twigma_scrape_rows_{START_ROW}_{END_ROW}.csv"
df.to_csv(OUTPUT_CSV_PATH, index=False)

print(f"\n✅ Results saved to: {OUTPUT_CSV_PATH}")

# === ✅ Summary statistics ===
total = len(df)
text_ok = df["text_success"].sum()
image_ok = df["image_success"].sum()
both_ok = ((df["text_success"]) & (df["image_success"])).sum()

def pct(x): return f"{100 * x / total:.2f}%"

print("\n📊 Summary:")
print(f"📝 Tweets with text successfully extracted: {text_ok}/{total} ({pct(text_ok)})")
print(f"🖼️  Tweets with image successfully downloaded: {image_ok}/{total} ({pct(image_ok)})")
print(f"✅ Tweets with both text and image: {both_ok}/{total} ({pct(both_ok)})")

def format_duration(seconds):
    mins, secs = divmod(int(seconds), 60)
    hrs, mins = divmod(mins, 60)
    return f"{hrs}h {mins}m {secs}s"

print(f"\n⏱️ Total runtime: {format_duration(duration)}({int(duration)}secs)")
if both_ok > 0:
    print(f"\n⏱️ Each fully successful process takes {duration / both_ok:.2f} seconds")
else:
    print("\n⚠️ No fully successful entries to calculate average time.")


✅ Results saved to: twigma_scrape_rows_1000_2000.csv

📊 Summary:
📝 Tweets with text successfully extracted: 172/568 (30.28%)
🖼️  Tweets with image successfully downloaded: 176/568 (30.99%)
✅ Tweets with both text and image: 172/568 (30.28%)

⏱️ Total runtime: 1h 0m 35s(3635secs)

⏱️ Each fully successful process takes 21.14 seconds


In [55]:
SUMMARY_PATH = OUTPUT_CSV_PATH.replace(".csv", "_summary.txt")

with open(SUMMARY_PATH, "w") as f:
    f.write(f"{SUMMARY_PATH}\n")
    f.write("📊 Summary:\n")
    f.write(f"📝 Tweets with text successfully extracted: {text_ok}/{total} ({pct(text_ok)})\n")
    f.write(f"🖼️  Tweets with image successfully downloaded: {image_ok}/{total} ({pct(image_ok)})\n")
    f.write(f"✅ Tweets with both text and image: {both_ok}/{total} ({pct(both_ok)})\n")
    f.write(f"⏱️ Total runtime: {format_duration(duration)} ({int(duration)}secs)\n")
    f.write(f"⏱️ Each fully successful process takes {duration / both_ok:.2f} seconds\n")

print(f"📝 Summary saved to: {SUMMARY_PATH}")

📝 Summary saved to: twigma_scrape_rows_1000_2000_summary.txt


### Downloading Files ###

In [56]:
# ✅ Zip image directory
zip_name = OUTPUT_IMAGE_DIR + ".zip"
shutil.make_archive(OUTPUT_IMAGE_DIR, 'zip', OUTPUT_IMAGE_DIR)

# ✅ Download files
files.download(zip_name)
files.download(OUTPUT_CSV_PATH)
files.download(SUMMARY_PATH)

print(f"📥 Download links generated for CSV and image archive.\n {zip_name} \n {OUTPUT_CSV_PATH} \n {SUMMARY_PATH}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Download links generated for CSV and image archive.
 /content/images_1000_2000.zip 
 twigma_scrape_rows_1000_2000.csv 
 twigma_scrape_rows_1000_2000_summary.txt


### Retry (Not recommend for Efficiency) ###

In [66]:
# RE_START_ROW = 0
# RE_END_ROW = 100  # Set your own range

# # === CONFIG ===
# RETRY_CSV_PATH = f"twigma_scrape_rows_{RE_START_ROW}_{RE_END_ROW}.csv"
# OUTPUT_IMAGE_DIR = f"images_{RE_START_ROW}_{RE_END_ROW}"
# os.makedirs(OUTPUT_IMAGE_DIR, exist_ok=True)

# options = uc.ChromeOptions()
# options.binary_location = "/usr/bin/google-chrome"
# options.add_argument("--headless=new")
# options.add_argument("--no-sandbox")
# options.add_argument("--disable-dev-shm-usage")
# driver = uc.Chrome(options=options)

# # === LOAD DATA ===
# df = pd.read_csv(RETRY_CSV_PATH)
# df_retry = df[(df["text_success"] == False) | (df["image_success"] == False)].copy()
# print(f"🔁 Total to retry: {len(df_retry)}")

# # === RECORD NEW RESULTS ===
# new_text_success = 0
# new_image_success = 0

# for idx, row in tqdm(df_retry.iterrows(), total=len(df_retry), desc="🔁 Retrying failed entries"):
#     tweet_id = str(row["tweet_id"])
#     tweet_url = f"https://twitter.com/i/web/status/{tweet_id}"
#     cdn_image_old = row["cdn_image_name"]

#     # Retry flag
#     got_text = False
#     got_image = False

#     try:
#         driver.get(tweet_url)
#         # time.sleep(0.5)

#         # Retry text if it failed
#         if row["text_success"] == False:
#             try:
#                 tweet_elem = WebDriverWait(driver, 3).until(
#                     EC.presence_of_element_located((By.XPATH, '//article//div[@data-testid="tweetText"]'))
#                 )
#                 tweet_text = tweet_elem.text.strip()
#                 df.loc[idx, "tweet_text"] = tweet_text
#                 df.loc[idx, "text_success"] = True
#                 new_text_success += 1
#                 got_text = True
#             except:
#                 pass

#         # Retry image if it failed
#         if row["image_success"] == False:
#             try:
#                 img_elem = WebDriverWait(driver, 1).until(
#                     EC.presence_of_element_located((By.XPATH, '//article//img[contains(@src, "pbs.twimg.com/media")]'))
#                 )
#                 media_url = img_elem.get_attribute("src")

#                 parsed = urlparse(media_url)
#                 cdn_filename = os.path.basename(parsed.path)
#                 if not re.search(r"\.(jpg|jpeg|png|webp)$", cdn_filename):
#                     cdn_filename += ".jpg"

#                 full_filename = f"{tweet_id}_{cdn_filename}"
#                 full_path = os.path.join(OUTPUT_IMAGE_DIR, full_filename)

#                 img_data = requests.get(media_url, timeout=5).content
#                 with open(full_path, "wb") as f:
#                     f.write(img_data)

#                 df.loc[idx, "cdn_image_name"] = cdn_filename
#                 df.loc[idx, "image_success"] = True
#                 new_image_success += 1
#                 got_image = True

#             except:
#                 pass

#     except Exception as e:
#         print(f"[FAIL] Tweet {tweet_id}: {e}")

# driver.quit()

# # === SAVE NEW MERGED RESULT ===
# REPAIRED_CSV_PATH = RETRY_CSV_PATH.replace(".csv", "_repaired.csv")
# df.to_csv(REPAIRED_CSV_PATH, index=False)

# # === STATS ===
# total_retry = len(df_retry)
# final_total = len(df)
# final_text = df["text_success"].sum()
# final_img = df["image_success"].sum()
# final_both = ((df["text_success"]) & (df["image_success"])).sum()

# def pct(x, total): return f"{100 * x / total:.2f}%"

# print("\n📊 Retry Summary:")
# print(f"📝 Newly recovered texts:  {new_text_success}/{total_retry} ({pct(new_text_success, total_retry)})")
# print(f"🖼️  Newly recovered images: {new_image_success}/{total_retry} ({pct(new_image_success, total_retry)})")

# print("\n📦 Overall Total After Merge:")
# print(f"📝 Total successful texts:  {final_text}/{final_total} ({pct(final_text, final_total)})")
# print(f"🖼️  Total successful images: {final_img}/{final_total} ({pct(final_img, final_total)})")
# print(f"✅ Fully successful entries: {final_both}/{final_total} ({pct(final_both, final_total)})")

# print(f"\n✅ Updated CSV saved to: {REPAIRED_CSV_PATH}")

🔁 Total to retry: 45


🔁 Retrying failed entries: 100%|██████████| 45/45 [03:29<00:00,  4.66s/it]



📊 Retry Summary:
📝 Newly recovered texts:  5/45 (11.11%)
🖼️  Newly recovered images: 5/45 (11.11%)

📦 Overall Total After Merge:
📝 Total successful texts:  21/61 (34.43%)
🖼️  Total successful images: 22/61 (36.07%)
✅ Fully successful entries: 21/61 (34.43%)

✅ Updated CSV saved to: twigma_scrape_rows_0_100_repaired.csv


In [69]:
# # ✅ Zip image directory
# zip_name = OUTPUT_IMAGE_DIR + ".zip"
# shutil.make_archive(OUTPUT_IMAGE_DIR, 'zip', OUTPUT_IMAGE_DIR)

# # ✅ Download files
# files.download(zip_name)
# files.download(REPAIRED_CSV_PATH)

# print("📥 Download links generated for Repaired-CSV and image archive.")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Download links generated for Repaired-CSV and image archive.
